<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

In [2]:
#Setup libraries
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input


<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [3]:
import requests
import zipfile
import io

# URL of the zipped dataset
url = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip"

# Send a GET request to download the file
response = requests.get(url)

if response.status_code == 200:
    # Create a file-like object from the downloaded content
    zip_file = io.BytesIO(response.content)

    # Extract the contents of the zip file to a directory
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall('')  # Replace '/path/to/extract/' with your desired extraction path
    print("Dataset downloaded and extracted successfully.")
else:
    print("Failed to download the dataset.")

Dataset downloaded and extracted successfully.


After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [4]:

from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [5]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [6]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [7]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [8]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [9]:
model = Sequential()
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))
model.add(Dense(num_classes, activation='softmax'))
model.layers[0].trainable = False

58889256/58889256 [==============================] - 2s 0us/step


In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

## The code below is commented out due to a switch in the runtime session type which resulted in this session being completed prior to the session which ran the other task. As such, the model created was saved and then reloaded in the second step.

In [12]:
"""fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)
"""

'fit_history = model.fit_generator(\n    train_generator,\n    steps_per_epoch=steps_per_epoch_training,\n    epochs=num_epochs,\n    validation_data=validation_generator,\n    validation_steps=steps_per_epoch_validation,\n    verbose=1,\n)\n'

In [13]:
"""model.save('classifier_VGG16_model.h5')

from google.colab import files

files.download('classifier_VGG16_model.h5')"""

"model.save('classifier_VGG16_model.h5')\n\nfrom google.colab import files\n\nfiles.download('classifier_VGG16_model.h5')"

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model.
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [21]:
#Loading both saved models. View explanation above

from keras.models import load_model
resnet = load_model('classifier_resnet_model.h5')
vgg = load_model('classifier_VGG16_model.h5')

In [23]:
#Creating the test generator
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    shuffle=False,
    class_mode='categorical')

Found 500 images belonging to 2 classes.


In [24]:
#The evaluate_generator method
# Evaluate model1 on the test set
evaluation_results_model1 = resnet.evaluate_generator(test_generator)

# Evaluate model2 on the test set
evaluation_results_model2 = vgg.evaluate_generator(test_generator)

# Display evaluation results
print("Evaluation Results for RESNET model:")
print("Loss:", evaluation_results_model1[0])
print("Accuracy:", evaluation_results_model1[1])

print("\nEvaluation Results for VGG16 model:")
print("Loss:", evaluation_results_model2[0])
print("Accuracy:", evaluation_results_model2[1])

<ipython-input-24-3177033a4b98>:3: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  evaluation_results_model1 = resnet.evaluate_generator(test_generator)
<ipython-input-24-3177033a4b98>:6: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  evaluation_results_model2 = vgg.evaluate_generator(test_generator)


Evaluation Results for RESNET model:
Loss: 0.002152025466784835
Accuracy: 1.0

Evaluation Results for VGG16 model:
Loss: 0.012740151025354862
Accuracy: 0.9980000257492065


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li>
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [25]:
# Predict classes for the first five images
num_images_to_predict = 5
predictions = resnet.predict_generator(test_generator, steps=num_images_to_predict)

# Get class indices from the generator
class_indices = test_generator.class_indices

# Get the class labels from the generator
labels = {v: k for k, v in class_indices.items()}

# Print predicted classes for the first five images
for i in range(num_images_to_predict):
    predicted_class = predictions[i].argmax()  # Get the index of the predicted class
    class_label = labels[predicted_class]  # Get the label corresponding to the predicted class index
    print(f"Predicted class for image {i+1}: {class_label}")

<ipython-input-25-1564ab37d16d>:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = resnet.predict_generator(test_generator, steps=num_images_to_predict)


Predicted class for image 1: negative
Predicted class for image 2: negative
Predicted class for image 3: negative
Predicted class for image 4: negative
Predicted class for image 5: negative


# Other Questions:

How many images were used for training the VGG16 model? (5 marks)

In [26]:
num_training_images = train_generator.n
print("Number of images used for training the VGG16 model:", num_training_images)

Number of images used for training the VGG16 model: 30001


How many parameters did the VGG16 pre-trained model have? (5 marks)
14,714,688.

In [27]:
vgg16_model = VGG16()
vgg16_model.summary()


553467096/553467096 [==============================] - 14s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                           

### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).